In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Создание сессии Spark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Data_exploration').getOrCreate()

# Чтение данных

In [11]:
# Вывод в консоль части данных

!head -n5 Data/cars.csv

manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,universal,False,owned,all,10900.0,False,Минская обл.,9,13,False,True,True,True,False,True,False,True,True,True,16
Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,universal,False,owned,all,5000.0,True,Минская обл.,12,54,False,True,False,False,True,True,False,False,False,True,83
Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,suv,False,owned,all,2800.0,True,Минская обл.,4,72,False,True,False,False,False,False,False,False,True,True,151
Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,sedan,False,owned,a

In [12]:
df = spark.read.format('csv').option('header', 'true').load('Data/cars.csv')
df

DataFrame[manufacturer_name: string, model_name: string, transmission: string, color: string, odometer_value: string, year_produced: string, engine_fuel: string, engine_has_gas: string, engine_type: string, engine_capacity: string, body_type: string, has_warranty: string, state: string, drivetrain: string, price_usd: string, is_exchangeable: string, location_region: string, number_of_photos: string, up_counter: string, feature_0: string, feature_1: string, feature_2: string, feature_3: string, feature_4: string, feature_5: string, feature_6: string, feature_7: string, feature_8: string, feature_9: string, duration_listed: string]

In [ ]:
# Структура данных - pyspark DataFrame

type(df)

In [13]:
# Вывод первых 5 строк полученного DataFrame

df.show(5)

+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|manufacturer_name|model_name|transmission| color|odometer_value|year_produced|engine_fuel|engine_has_gas|engine_type|engine_capacity|body_type|has_warranty|state|drivetrain|price_usd|is_exchangeable|location_region|number_of_photos|up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|duration_listed|
+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+----

В силу большого числа признаков в рассматриваемых данных - вывод нечитаемый. 

**Иной метод вывода данных:**

In [14]:
# Выводим две строки в вертикальном просмотре

df.show(2, vertical=True)

-RECORD 0-------------------------
 manufacturer_name | Subaru       
 model_name        | Outback      
 transmission      | automatic    
 color             | silver       
 odometer_value    | 190000       
 year_produced     | 2010         
 engine_fuel       | gasoline     
 engine_has_gas    | False        
 engine_type       | gasoline     
 engine_capacity   | 2.5          
 body_type         | universal    
 has_warranty      | False        
 state             | owned        
 drivetrain        | all          
 price_usd         | 10900.0      
 is_exchangeable   | False        
 location_region   | Минская обл. 
 number_of_photos  | 9            
 up_counter        | 13           
 feature_0         | False        
 feature_1         | True         
 feature_2         | True         
 feature_3         | True         
 feature_4         | False        
 feature_5         | True         
 feature_6         | False        
 feature_7         | True         
 feature_8         |

# DataFrame API

**`.select()`** для выбора колонок аналогично `sql`

In [16]:
df.select('manufacturer_name', 'model_name', 'transmission').show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|           Subaru|   Outback|   automatic|
|           Subaru|   Outback|   automatic|
|           Subaru|  Forester|   automatic|
|           Subaru|   Impreza|  mechanical|
|           Subaru|    Legacy|   automatic|
+-----------------+----------+------------+
only showing top 5 rows



In [17]:
df.select(df['manufacturer_name'], df['model_name'], df['transmission']).show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|           Subaru|   Outback|   automatic|
|           Subaru|   Outback|   automatic|
|           Subaru|  Forester|   automatic|
|           Subaru|   Impreza|  mechanical|
|           Subaru|    Legacy|   automatic|
+-----------------+----------+------------+
only showing top 5 rows



In [26]:
import pyspark.sql.functions as F

df.select(F.col('manufacturer_name'), F.col('model_name'), F.col('transmission')).show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|           Subaru|   Outback|   automatic|
|           Subaru|   Outback|   automatic|
|           Subaru|  Forester|   automatic|
|           Subaru|   Impreza|  mechanical|
|           Subaru|    Legacy|   automatic|
+-----------------+----------+------------+
only showing top 5 rows



**`.filter()`** - фильтрация данных

In [60]:
df\
.select("manufacturer_name", "model_name", "transmission")\
.filter(F.col("manufacturer_name") == "Audi")\
.filter(F.col("transmission") != "mechanical")\
.show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|        Q7|   automatic|
|             Audi|        TT|   automatic|
|             Audi|        A6|   automatic|
|             Audi|        Q3|   automatic|
|             Audi|        Q5|   automatic|
+-----------------+----------+------------+
only showing top 5 rows



In [63]:
df\
.select(F.col('manufacturer_name'), F.col('model_name'), F.col('transmission'))\
.filter((F.col('transmission') == 'automatic') & (F.col('manufacturer_name') == 'Subaru'))\
.show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|           Subaru|   Outback|   automatic|
|           Subaru|   Outback|   automatic|
|           Subaru|  Forester|   automatic|
|           Subaru|    Legacy|   automatic|
|           Subaru|   Outback|   automatic|
+-----------------+----------+------------+
only showing top 5 rows



In [64]:
df.select('manufacturer_name').filter(F.col('manufacturer_name') != 'Volkswagen').distinct().show()

+-----------------+
|manufacturer_name|
+-----------------+
|            Lexus|
|           Jaguar|
|            Rover|
|           Lancia|
|             Jeep|
|       Mitsubishi|
|              Kia|
|             Mini|
|            Lifan|
|             LADA|
|        SsangYong|
|             Audi|
|             Seat|
|         Cadillac|
|          Москвич|
|       Alfa Romeo|
|            Geely|
|          Renault|
|           Daewoo|
|            Acura|
+-----------------+
only showing top 20 rows



**`.count()`** подсчёт строк

In [65]:
# Всего записей

df.count()

38531

In [69]:
# Уникальные строки

df.select('manufacturer_name').distinct().count()

55

In [71]:
df.select('transmission').distinct().count()

2

**`.groupBy()`** и **`.orderBy()`** - группировка и сортировка данных

In [76]:
df.groupBy('manufacturer_name').count().show()

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|            Lexus|  213|
|           Jaguar|   53|
|            Rover|  235|
|           Lancia|   92|
|             Jeep|  107|
|       Mitsubishi|  887|
|              Kia|  912|
|             Mini|   68|
|            Lifan|   47|
|             LADA|  146|
|        SsangYong|   79|
|             Audi| 2468|
|             Seat|  303|
|         Cadillac|   43|
|          Москвич|   55|
|       Alfa Romeo|  207|
|            Geely|   71|
|          Renault| 2493|
|           Daewoo|  221|
+-----------------+-----+
only showing top 20 rows



In [72]:
df.columns

['manufacturer_name',
 'model_name',
 'transmission',
 'color',
 'odometer_value',
 'year_produced',
 'engine_fuel',
 'engine_has_gas',
 'engine_type',
 'engine_capacity',
 'body_type',
 'has_warranty',
 'state',
 'drivetrain',
 'price_usd',
 'is_exchangeable',
 'location_region',
 'number_of_photos',
 'up_counter',
 'feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'duration_listed']